# Feature Engineering
In Feature Engineering, we'll transform raw game stats into smart features that help predict fantasy scores. This includes:

  - Rolling averages - Player's last 5-10 game performance
  - Matchup data - How players perform vs specific teams/defenses
  - Schedule factors - Rest days, back-to-back games, home/away
  - Recent form - Is the player trending up or down?
  - Opponent strength - Defensive rankings and pace of play

  The goal is to give our model the same context fantasy experts use - like knowing LeBron scores less on
  back-to-backs or Steph shoots better at home. These features capture patterns beyond basic box scores.